## Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import requests

import calendar
from datetime import datetime, date, timedelta

import re

from tqdm.notebook import tqdm


## Краткое описание

Итак, согласно  **HeadHunter API** документации:
- [Github](https://github.com/hhru/api) документация
- [OpenAPI](https://api.hh.ru/openapi/redoc) документация

У нас есть несколько методов просмотра и поиска интересующих нас вакансий по сайту , но для наших целей я предлагаю обратить внимание на следующие:

1. **Поиск по вакансиям**
	- `GET /vacancies` вернёт результаты поиска вакансий.
	- [Github](https://github.com/hhru/api/blob/master/docs/vacancies.md#search) документация
    - [OpenAPI](https://api.hh.ru/openapi/redoc#tag/Poisk-vakansij) документация

1. **Просмотр вакансии**
	 - `GET /vacancies/{vacancy_id}` где `vacancy_id` – идентификатор вакансии. Вернёт подробную информацию по указанной вакансии.
	 - [Github](https://github.com/hhru/api/blob/master/docs/vacancies.md#item) документация
     
1. **Поиск по компаниям**
	- `GET /employers`
	- [OpenAPI](https://api.hh.ru/openapi/redoc#tag/Rabotodatel/operation/search-employer) документация

1. **Получение информации о компании**
	 - `GET /employers/{employer_id}`
	- [OpenAPI] (https://api.hh.ru/openapi/redoc#tag/Rabotodatel/operation/get-employer-info) документация

Исследуем данные методы

## Блок обьявления переменных

### Определимся с наименованием должностей для поискового запроса

Согласно  **HeadHunter API** документации:
- [Github](https://github.com/hhru/api/blob/master/docs/vacancies.md#search) документация
- [OpenAPI](https://api.hh.ru/openapi/redoc#tag/Poisk-vakansij) документация

Чтобы нам осуществить поиск вакансии по должности нам необходимо в GET параметрах запроса указать:
-   `text` — текстовое поле.  
    Переданное значение ищется в полях вакансии, указанных в параметре `search_field`.
    
    [Доступен язык запросов](https://hh.ru/article/1175). 
    Специально для этого поля есть [автодополнение](https://github.com/hhru/api/blob/master/docs/suggests.md#vacancy-search-keyword).

### Рассмотрим возвожные варианты наших поисковых запросов

In [ ]:
# 1-ая тестовая вариациия поискового запроса
job_1 = 'Курьер' # 'Аналитик данных'
job_1

In [ ]:
# 2-ая тестовая вариациия поискового запроса
job_2 = 'NAME:(Аналитик OR Analyst)'
job_2

In [ ]:
# 3-ая тестовая вариациия поискового запроса
job_3 = 'COMPANY_NAME:Самокат'
job_3

In [ ]:
# список тестовых вариаций поискового запроса
job_title = ['Аналитик данных',
             '"!Аналитик BI"',
             'Data Analyst',
             'Аналитик*',
             '"Аналитик" OR "Analyst"',
             'COMPANY_NAME:  "Самокат (ООО Умный ритейл)"',
             'NAME:(Аналитик OR Analyst) and COMPANY_NAME:Headhunter',
             'NAME:(Аналитик OR Analyst) and COMPANY_NAME:Самокат',
             'NAME:(Аналитик OR Analyst OR "Data" OR Курьер)',
             'NAME:(Аналитик OR "Data Analyst" OR "Инженер данных" OR "Data engineer" OR "Дата саентист" OR "Data Scientist")',
             'NAME:(Аналитик OR Курьер OR Инженер)'
            ]
job_title

In [ ]:
# как вариант выбора из нашего списка тестовых вариаций поискового запроса
job_title[3]

In [ ]:
# Поиск по следующим параметрам должности
job_search_field = job_title[-1]
job_search_field

### Сформируем даты

#### Сегодня 

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
today

In [ ]:
today_day_sec = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
today_day_sec

In [ ]:
# correct, ISO-8601 (but not UTC)
datetime.now().isoformat(timespec='seconds')

#### Вчера

In [ ]:
yesterday = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
yesterday

In [ ]:
yesterday_sec = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S')
yesterday_sec

#### Год


In [ ]:
year = date.today().year
year

#### Месяц

In [ ]:
month = date.today().month
month

#### День

In [ ]:
day = date.today().day
day

#### Календарь

In [ ]:
# узнать количество дней в месяце
calendar.monthrange(year,month)

In [ ]:
# напечатаем календарь
calendar.prmonth(year,month)

#### Варианты формирования промежутков времени

In [ ]:
#с интервалом по дням
pd.date_range(start='01/01/2023', end='01/31/2023')

In [ ]:
# промежутки с интервалом по дням вперед
pd.date_range(start='02/01/2023', periods=15)

In [ ]:
# промежутки с интервалом по дням назад
pd.date_range(end='02/01/2023', periods=15)

In [ ]:
# узнаем когда конец месяца
pd.date_range(start='02/01/2023',freq='M', periods=1)

In [ ]:
# промежуток с начала текущего месяца
current_month_time_period = pd.date_range(end=today, periods= int(day))
current_month_time_period

In [ ]:
current_month_time_period.strftime('%Y-%m-%d').tolist()

In [ ]:
current_month_date_range = []
for time in current_month_time_period:
    current_month_date_range.append(pd.date_range(time,freq='D', periods=1).strftime('%Y-%m-%d').tolist())
current_month_date_range

In [ ]:
for date in current_month_date_range:
    end_date_to = date[0]
    print ('Date to:', end_date_to)
    start_date_from = (pd.to_datetime(end_date_to) - timedelta(days=1)).strftime('%Y-%m-%d')
    print ('Date from:',start_date_from)

In [ ]:
# промежуток времени в часах
pd.date_range(start='02/10/2023',freq='H', periods=24)

In [ ]:
# промежуток времени в часах
hours_date_range_period = pd.date_range(start='02/13/2023',freq='H', periods=24)
hours_date_range_period

In [ ]:
# промежуток времени в часах
hours_date_range= []
for time in hours_date_range_period:
    hours_date_range.append(pd.date_range(time,freq='D', periods=1).strftime('%Y-%m-%dT%H:%M:%S').tolist())
hours_date_range

In [ ]:
# промежуток времени в часах
for hour in hours_date_range:
    start_hour = hour[0]
    print ('Date from:', start_hour)
    end_hour = (pd.to_datetime(start_hour) + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S')
    print ('Date to:',end_hour)

### Cформируем запрос за несколько дней с начала месяца

In [ ]:
# Количество страниц в запросе
number_of_pages = 100 # Количество элементов (по умолчанию - 10, максимальное значение - 100)

In [ ]:
# Адрес запроса вакансий
url_HH_vacancies = 'https://api.hh.ru/vacancies'

In [ ]:
job_2

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
# 
query_parametrs_0 = {
    'per_page': number_of_pages,
    'text': job_2,
    'area':'113', # Регион Россия
    'only_with_salary': True, # Показывать вакансии только с указанием зарплаты. 
    'responses_count_enabled': True, # Количеством откликов для вакансии включено
}

In [ ]:
vacancies_0 = pd.DataFrame()

# подсчет суммарного количество найденных вакансий за весь период поиска
sum_found = []

for date in tqdm(current_month_date_range):
    end_date_to = date[0]
    start_date_from = (pd.to_datetime(end_date_to) - timedelta(days=1)).strftime('%Y-%m-%d')
   
    #### наш запрос    
    page = 0
    for page_number in tqdm(range(0, 20)):
        # 20page x 100 per_page HeadHunter не выдает в запросе не больше 2000 ?
        query_parametrs_0['page'] = page_number
        query_parametrs_0['date_from'] = start_date_from
        query_parametrs_0['date_to'] = end_date_to
        request = requests.get(url=url_HH_vacancies,
                               params=query_parametrs_0)
        response = request.json()
        
        vacancies_0 = pd.concat([vacancies_0, pd.DataFrame(response['items'])])
   
    # количество найденных вакансий за каждый день
    found= response['found']
    # суммарное количество найденных вакансий за весь период
    sum_found.append(found)

In [ ]:
vacancies_0


In [ ]:
# количество найденых вакансий по дням
# ВАЖНО - убедись что каждый день меньше 2000
sum_found

In [ ]:
# суммарное количество найденных вакансий за весь период
# ВНИМАНИЕ - должно совпасть с количеством строк в датафрейме выше
sum(sum_found)

In [ ]:
# преобразуем время и дату
vacancies_0['published_at'] = pd.to_datetime(vacancies_0['published_at'])
vacancies_0['created_at'] = pd.to_datetime(vacancies_0['created_at'])
vacancies_0['created_at_str'] = pd.to_datetime(vacancies_0['created_at']).dt.strftime('%Y-%m-%d')

In [ ]:
vacancies_0['created_at_str'].describe()

In [ ]:
vacancies_0\
    .groupby('created_at_str', as_index=False)\
    .agg({'id':'count'})\
    .sort_values('id', ascending = False)

In [ ]:
vacancies_0_for_plot = vacancies_0\
    .groupby('created_at_str', as_index=False)\
    .agg({'id':'count'})

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(data=vacancies_0_for_plot, x="created_at_str", y="id")
locs, labels = plt.xticks()
plt.setp(labels, rotation=90);

In [ ]:
calendar.prmonth(2023, 2)

In [ ]:
vacancies_0.describe()

In [ ]:
vacancies_0.id.describe()

In [ ]:
# посмотрим какие у нас ID повторяются ?
# может дело в 00:00 ?
vacancies_0\
    .groupby('id', as_index=False)\
    .agg({'premium':'count'})\
    .sort_values('id', ascending = True)

In [ ]:
# явной закономерности повторяния быстро не увидел - нужно проверить позже
vacancies_0\
    .query ('id == ["76515203","37450190","37644107","40209605"]')\
    .sort_values('id', ascending = True)\
    [['id','created_at_str','created_at','published_at']]
    

### Справочники поработаем с ними


#### Дерево всех регионов
Согласно  **HeadHunter API** документации:
- [Справочник регионов](https://github.com/hhru/api/blob/master/docs/areas.md)
- `GET /areas` возвращает древовидный список всех регионов с указанием названия региона, его идентификатором и ссылкой на родительский регион `parent_id`.

In [ ]:
# Адрес запроса дерева регионов
url_HH_areas = 'https://api.hh.ru/areas'

In [ ]:
areas = requests.get(url_HH_areas).json()
areas

#### Справочник стран
Согласно  **HeadHunter API** документации:
- [Справочник регионов](https://github.com/hhru/api/blob/master/docs/areas.md)
- `GET /areas/countries` вернёт подмножество регионов, являющихся странами.

In [ ]:
# Адрес запроса дерева регионов
url_HH_countries = 'https://api.hh.ru/areas/countries'

In [ ]:
# Именно отсюда мы понимаем что для России выбираем 113
countries = requests.get(url_HH_countries).json()
countries

#### Словари


In [ ]:
# Адрес запроса 
url_HH_dictionaries = 'https://api.hh.ru/dictionaries'

In [ ]:
dictionaries = requests.get(url_HH_dictionaries).json()
dictionaries

In [ ]:
# Вот отсюда языковой поисковый запрос на HH берет необходимые ключи
# В поисковом запросе мы также можем при построении сложных запросов использовать синтаксис
# Список доступных полей:

# !ID - id вакансии (восклицательный знак обязателен)
# NAME - название вакансии
# !COMPANY_ID - id компании (восклицательный знак обязателен)
# COMPANY_NAME - название компании
# DESCRIPTION - описание вакансии

# NAME:(python OR java) and COMPANY_NAME:Headhunter

dictionaries['vacancy_search_fields']

#### Специализации
Согласно  **HeadHunter API** документации:
- [Специализации](https://github.com/hhru/api/blob/master/docs/specializations.md)
- `GET /specializations` возвращает справочник всех профессиональных областей и специализаций.

In [ ]:
# Адрес запроса 
url_HH_specializations = 'https://api.hh.ru/specializations'

In [ ]:
specializations = requests.get(url_HH_specializations).json()
specializations

### Попробуем запрос по компаниям 

1. **Поиск по компаниям**
	- `GET /employers`
	- [OpenAPI](https://api.hh.ru/openapi/redoc#tag/Rabotodatel/operation/search-employer) документация


In [ ]:
# Адрес запроса поиска по компаниям 
url_HH_employers = 'https://api.hh.ru/employers'

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
query_parametrs_employers = {
    'per_page': number_of_pages,
    'text': 'Газпром' # Текст для поиска. Переданное значение ищется в названии и описании работодателя                               
                            }

In [ ]:
requests_employers = requests.get(url_HH_employers,
                        params= query_parametrs_employers)
response_employers = requests_employers.json()

In [ ]:
# посмотрим какие у нас есть ключи
response_employers.keys()

In [ ]:
# посмотрим какие у нас есть ключи
response_employers['items'][0].keys()

In [ ]:
response_employers

In [ ]:
response_employers ['items'][0]

In [ ]:
response_employers ['items'][0]['id']


In [ ]:
response_employers ['items'][0]['name']

In [ ]:
# количество результатов поиска
response_employers ['found']

#### Получим подробную информацию о конкретной компании

In [ ]:
first_employer_id = response_employers ['items'][0]['id']
first_employer_id

In [ ]:
# Адрес запроса поиска по компаниям 
url_HH_employer = f'https://api.hh.ru/employers/{first_employer_id}'
url_HH_employer

In [ ]:
requests_unique_employer = requests.get(url_HH_employer)
response_unique_employer = requests_unique_employer.json()

In [ ]:
response_unique_employer

In [ ]:
# Ищем страницу на сайте HH и в url берем наш ID (https://hh.ru/employer/3388)
# Ну или берем из нашего предыдущего поиска по компаниям

employer_id = 3388 #ГазпромБанк
#employer_id = 1740 #Яндекс

In [ ]:
# Адрес запроса поиска вакансий по конкретной компании
url_HH_employer_vacancies = f'https://api.hh.ru/vacancies?employer_id={employer_id}'
url_HH_employer_vacancies

In [ ]:
requests_employer_vacancies = requests.get(url_HH_employer_vacancies)
response_employer_vacancies = requests_employer_vacancies.json()

In [ ]:
response_employer_vacancies

In [ ]:
# Количество найденных вакансий
response_employer_vacancies ['found']

In [ ]:
# количество страниц
response_employer_vacancies ['pages']

In [ ]:
# сколько вакансий на страницу
response_employer_vacancies ['per_page']

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
# 
query_parametrs_employer_vacancies = {
    'per_page': number_of_pages,
    'text': job_2,
    'area':'113', # Регион Россия
    'only_with_salary': True, # Показывать вакансии только с указанием зарплаты. 
    'responses_count_enabled': True, # Количеством откликов для вакансии включено
}

In [ ]:
# список со всей информацией
employer_vacancies = []

# датафрейм только с вакансиями
df_employer_vacancies = pd.DataFrame()

for i in tqdm(range(0, (response_employer_vacancies ['pages']))):
    # работаем со списком
    employer_vacancies.append(requests.get (url_HH_employer_vacancies,
                                            params={
                                                'page': i, 
                                                'per_page': (response_employer_vacancies ['per_page'])
                                                    }).json())
    # работаем с датафреймом
    df_employer_vacancies = pd.concat([df_employer_vacancies, pd.DataFrame((requests.get (url_HH_employer_vacancies,
                                            params={
                                                'page': i, 
                                                'per_page': (response_employer_vacancies ['per_page'])
                                                    }).json())['items'])])

In [ ]:
# ВНИМАНИЕ большая выдача - все страницы в один лист!
employer_vacancies

In [ ]:
#  превратим полный список в pandas датафрейм
df_employer_vacancies_full = pd.DataFrame(employer_vacancies)

In [ ]:
# вот резудьтаты всеъ запросов по страницам
df_employer_vacancies_full

In [ ]:
# результат запроса первой страницы
df_employer_vacancies_full['items'][0]

In [ ]:
#  посмотрим датафрейм только с вакансиями
df_employer_vacancies

## Реализация обработки кода ответа от API

In [ ]:
# пример хорошего удачного запроса
# на примере из предыдущего запроса
good_URL = f'https://api.hh.ru/vacancies?employer_id={employer_id}'
good_URL

In [ ]:
# пример неудачного запроса
# на примере из предыдущего запроса
# не полный запрос не хватает параметров !!!
bad_URL = f'https://api.hh.ru/vacancies?employer_id='
bad_URL

In [ ]:
# пример хорошего удачного запроса
requests_good = requests.get(good_URL)

In [ ]:
# пример неудачного запроса
requests_bad = requests.get(bad_URL)

In [ ]:
# пример хорошего удачного запроса
#Смотрим какой код ответа
requests_good.status_code

In [ ]:
# пример неудачного запроса
#Смотрим какой код ответа
requests_bad.status_code

In [ ]:
# пример хорошего удачного запроса
#Смотрим какой код ответа
requests_good

In [ ]:
# пример неудачного запроса
#Смотрим какой код ответа
requests_bad

In [ ]:
# пример хорошего удачного запроса
# булевое сравнение что код ОК
requests_good.status_code == requests.codes.ok

In [ ]:
# пример неудачного запроса
# булевое сравнение что код ОК
requests_bad.status_code == requests.codes.ok

In [ ]:
# пример хорошего удачного запроса
# посмотри что будет происходить при неудачном запросе 4хх или 5хх
requests_good.raise_for_status()

In [ ]:
# пример неудачного запроса
# посмотри что будет происходить при неудачном запросе 4хх или 5хх
requests_bad.raise_for_status()

In [ ]:
# пример хорошего удачного запроса
# заголовок запроса
requests_good.headers

In [ ]:
# пример неудачного запроса
# заголовок запроса
requests_bad.headers

In [ ]:
# пример хорошего удачного запроса
# кодировка
requests_good.encoding

In [ ]:
# пример неудачного запроса
# кодировка
requests_bad.encoding

In [ ]:
# пример хорошего удачного запроса
# текст ответа сайта
requests_good.text

In [ ]:
# пример неудачного запроса
# текст ответа сайта
requests_bad.text

In [ ]:
# пример хорошего удачного запроса
# текст ответа сайта
requests_good.json()

In [ ]:
# пример неудачного запроса
# текст ответа сайта
requests_bad.json()

## Сформируем наш запрос где попробуем добиться ограничения в 2000 вакансий

In [ ]:
# Количество страниц в запросе
number_of_pages

In [ ]:
# Адрес запроса вакансий
url_HH_vacancies

In [ ]:
job_search_field

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
# 
query_parametrs_1 = {
    'per_page': number_of_pages,
    'text': job_search_field,
    'area':'113', # Регион Россия
    'only_with_salary': True, # Показывать вакансии только с указанием зарплаты.
    'date_from': '2023-01-25',  #фильтр по дате публикациии (от)
    'responses_count_enabled': True, # Количеством откликов для вакансии включено
}

In [ ]:
request_1 = requests.get(url=url_HH_vacancies,
                           params=query_parametrs_1)

response_1 = request_1.json()

In [ ]:
response_1

In [ ]:
response_1 ['found']

In [ ]:
response_1 ['pages']

In [ ]:
response_1 ['per_page']

In [ ]:
# на каком количестве страниц уместиться все ?
response_1 ['found'] / response_1 ['per_page']

In [ ]:
vacancies_1_list = []
for i in tqdm(range(0, 35)):
    vacancies_1_list.append(requests.get(url=url_HH_vacancies,
                               params= {
                                        'page': i,
                                        'per_page': number_of_pages, 
                                        'text': job_search_field,  
                                        'area':'113', # Регион Россия 
                                        'only_with_salary': True, # Показывать вакансии только с указанием зарплаты. 
                                        'date_from': '2023-01-25',  #фильтр по дате бубликациии(от)
                                        'responses_count_enabled': True, # Количеством откликов для вакансии включено
                                        }
                               ).json())

In [ ]:
vacancies_1_list

In [ ]:
#  превратим все в наш датафрейм
df_vacancies_1 = pd.DataFrame(vacancies_1_list)

In [ ]:
# ну вот встречаем ошибку что мы не можем смотреть больше 2000 шт
# you can't look up more than 2000 items in the list

df_vacancies_1

In [ ]:
# попробуем обойти запрет на пагинацию страниц смещением страниц на 15 к примеру
vacancies_2_list = []
for i in tqdm(range(0, 35)):
    vacancies_2_list.append(requests.get(url=url_HH_vacancies,
                               params= {
                                        'page': 15+i,
                                        'per_page': number_of_pages, 
                                        'text': job_search_field,  
                                        'area':'113', # Регион Россия 
                                        'only_with_salary': True, # Показывать вакансии только с указанием зарплаты. 
                                        'date_from': '2023-01-25',  #фильтр по дате бубликациии(от)
                                        'responses_count_enabled': True, # Количеством откликов для вакансии включено
                                        }
                               ).json())

In [ ]:
#  превратим все в наш датафрейм
df_vacancies_2 = pd.DataFrame(vacancies_2_list)

In [ ]:
# ограничение на 2000 запросов все еще тут
df_vacancies_2

#### понимаем что какая то ерунда и просто делаем часовое обновление и сращиваем все в один датафрейм

- `date_from` – дата, которая ограничивает снизу диапазон дат публикации вакансий.
Нельзя передавать вместе с параметром period.
Значение указывается в формате ISO 8601 - YYYY-MM-DD или **с точность до секунды YYYY-MM-DDThh:mm:ss±hhmm**.
Указанное значение будет округлено до ближайших 5 минут.

- `date_to` – дата, которая ограничивает сверху диапазон дат публикации вакансий.
Необходимо передавать только в паре с параметром date_from.
Нельзя передавать вместе с параметром period.
Значение указывается в формате ISO 8601 - YYYY-MM-DD или **с точность до секунды YYYY-MM-DDThh:mm:ss±hhmm**.
Указанное значение будет округлено до ближайших 5 минут.

In [ ]:
# Количество страниц в запросе
number_of_pages

In [ ]:
# Адрес запроса вакансий
url_HH_vacancies

In [ ]:
job_title[-1]

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
# 
query_parametrs_2 = {
    'per_page': number_of_pages,
    'text': job_title[-1],
    'area':'113', # Регион Россия
    'only_with_salary': True, # Показывать вакансии только с указанием зарплаты
    'responses_count_enabled': True, # Количеством откликов для вакансии включено
}

In [ ]:
vacancies_2 = pd.DataFrame()

# промежуток времени в часах
for hour in tqdm(hours_date_range):
    start_hour = hour[0]
    end_hour = (pd.to_datetime(start_hour) + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S')

    
    for page_number in tqdm(range(0, 20)):
        query_parametrs_2['page'] = page_number
        query_parametrs_2['date_from'] = start_hour
        query_parametrs_2['date_to'] = end_hour

        request_2 = requests.get(url=url_HH_vacancies,
                           params=query_parametrs_2)
    
        response_2 = request_2.json()
        
        #HH_vac.append(response_HH)
        vacancies_2 = pd.concat([vacancies_2, pd.DataFrame(response_2['items'])])

In [ ]:
vacancies_2

In [ ]:
vacancies_2.describe()

In [ ]:
#vacancies_2.describe(include=[object]) 

In [ ]:
#vacancies_2.describe(exclude=[np.number])  

In [ ]:
# vacancies_2\
#     .groupby('name', as_index=False)\
#     .agg({'id':'count'})\
#     .sort_values('id', ascending = False)\
#     .rename(columns={"name": "наименование вакансии", 
#                      "id": "количество"})\
#     .head(30)

## Сделаем простой запрос на общее количество вакансий

### Наш тестовый запрос № 1

In [ ]:
# Количество страниц в запросе
number_of_pages

In [ ]:
# Адрес запроса вакансий
url_HH_vacancies

In [ ]:
job_2

In [ ]:
# Подробнее параметры запроса смотри в описание API HeadHunter
query_parametrs_test_1 = {
    'per_page': number_of_pages,
    'text': job_2, # job_search_field как вариант
    'area':'113', # Регион Россия
    'only_with_salary': True, # Показывать вакансии только с указанием зарплаты.
    'date_from':  yesterday, 
    'date_to':  today,
    'responses_count_enabled': True, # Количеством откликов для вакансии включено
}

In [ ]:
vacancies_test_1 = pd.DataFrame()

page_number = 0
for page_number in tqdm(range(0, 20)):
    # 20page x 100 per_page 
    query_parametrs_test_1['page'] = page_number

    request_test_1 = requests.get(url=url_HH_vacancies,
                               params=query_parametrs_test_1)
    response_test_1 = request_test_1.json()
    
    vacancies_test_1 = pd.concat([vacancies_test_1, pd.DataFrame(response_test_1['items'])])

In [ ]:
# Количество найденных вакансий - должно совпадать с количеством строк в датафрейме
response_test_1 ['found']

In [ ]:
vacancies_test_1

In [ ]:
# посмотрим какие у нас есть ключи
response_test_1.keys()

In [ ]:
list(response_test_1.keys())

In [ ]:
response_test_1

In [ ]:
# response_test_1['items'][0].keys()

In [ ]:
vacancies_test_1.describe()

In [ ]:
vacancies_test_1.columns

In [ ]:
list(vacancies_test_1.columns)

### попробуем парсить данные нормально из первого запроса


In [ ]:
vacancies_test_1.head()

In [ ]:
vacancies_test_1.dtypes

In [ ]:
vacancies_test_1.columns

In [ ]:
vacancies_test_1_columns_list = vacancies_test_1.columns.tolist()
vacancies_test_1_columns_list

In [ ]:
vacancies_test_1.head(1)

In [ ]:
# Получим кортеж с именем столбца и содержимым строк для каждой строки нашего датафрейма:
for (ColumnName, RowData) in vacancies_test_1.head(1).iteritems():
    print('Имя столбца: \t\t', ColumnName)
    print('Содержимое строки \t\t:', RowData.values)

In [ ]:
# сделаем наш датафрейм короче, а то не особо удобно =)
df = vacancies_test_1.copy()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.equals.html
df.equals(vacancies_test_1)

#### поразбираемся с колонками

##### город

In [ ]:
df.area.iloc[0]

In [ ]:
df['city_id'] = df['area'].apply(lambda x: x['id'])
df['city'] = df['area'].apply(lambda x: x['name'])
df['city_url'] = df['area'].apply(lambda x: x['url'])

In [ ]:
df[['city_id','city','city_url']]

##### зарплата

In [ ]:
df.salary.iloc[0]

In [ ]:
df['salary_from'] = df['salary'].apply(lambda x: x['from'])
df['salary_to'] = df['salary'].apply(lambda x: x['to'])
df['salary_currency'] = df['salary'].apply(lambda x: x['currency'])
df['gross'] = df['salary'].apply(lambda x: x['gross'])

In [ ]:
df[['salary_from','salary_to','salary_currency','gross']]

##### информация о компании

In [ ]:
df.employer.iloc[0]

In [ ]:
df['employer_name'] = df['employer'].apply(lambda x: x['name'])

In [ ]:
df['employer_id'] = df['employer'].apply(lambda x: x['id'])
df['employer_url'] = df['employer'].apply(lambda x: x['alternate_url'])

In [ ]:
df[['employer_id','employer_name','employer_url']]

##### ВНИМАНИЕ!  Дополнительные текстовые отрывки
тут API реализованно так что отдает нам только часть актуальной информации

`snippet` (Дополнительные текстовые отрывки)

`requirement`	
**Отрывок** из требований по вакансии, если они найдены в тексте описания

`responsibility`	
**Отрывок** из обязанностей по вакансии, если они найдены в тексте описания

In [ ]:
df.snippet.iloc[0]

In [ ]:
df['requirement'] = df['snippet'].apply(lambda x: x['requirement'])
df['responsibility'] = df['snippet'].apply(lambda x: x['responsibility'])

In [ ]:
df[['requirement','responsibility']]

##### график работы

In [ ]:
df.schedule.iloc[0]

In [ ]:
df['working_time_id'] = df['schedule'].apply(lambda x: x['id'])
df['working_time_name'] = df['schedule'].apply(lambda x: x['name'])

In [ ]:
df[['working_time_id','working_time_name']]

##### число откликов

In [ ]:
df.counters.iloc[0]

In [ ]:
df['responses_counters'] = df['counters'].apply(lambda x: x['responses'])

In [ ]:
df[['responses_counters']]

##### професииональные роли

In [ ]:
df.professional_roles.iloc[0]

In [ ]:
df.professional_roles.iloc[0][0]

In [ ]:
df['professional_roles_id'] = df['professional_roles'].apply(lambda x: x[0]['id'])
df['professional_roles_name'] = df['professional_roles'].apply(lambda x: x[0]['name'])

In [ ]:
df[['professional_roles_id','professional_roles_name']]

#### приведем даты в порядок

In [ ]:
df.dtypes

In [ ]:
df['published_at'] = pd.to_datetime(df['published_at'])
df['created_at'] = pd.to_datetime(df['created_at'])

In [ ]:
df.published_at.dtypes

In [ ]:
df.published_at.min()

In [ ]:
df.published_at.max()

In [ ]:
columns_list_df = df.columns.tolist()
columns_list_df

In [ ]:
   df_final =        df[['id',
                         'premium',
                         'name',
                         'department',
                         'has_test',
                         'response_letter_required',
                         #'area',
                         #'salary',
                         #'type',
                         #'address',
                         'response_url',
                         'sort_point_distance',
                         'published_at',
                         'created_at',
                         'archived',
                         'apply_alternate_url',
                         'insider_interview',
                         'url',
                         'adv_response_url',
                         'alternate_url',
                         #'relations',
                         #'employer',
                         #'snippet', # вытаскиваем только часть... А надо ли полностью ?
                         'contacts',
                         #'schedule',
                         #'counters',
                         'working_days',
                         'working_time_intervals',
                         'working_time_modes',
                         'accept_temporary',
                         #'professional_roles',
                         'accept_incomplete_resumes',
                         'city_id',
                         'city',
                         'city_url',
                         'salary_from',
                         'salary_to',
                         'salary_currency',
                         'gross',
                         #'employer_id',
                         'employer_name',
                         #'employer_url',
                         'requirement',
                         'responsibility',
                         'working_time_id',
                         'working_time_name',
                         'responses_counters',
                         'professional_roles_id',
                         'professional_roles_name']].copy()

In [ ]:
# Получим кортеж с именем столбца и содержимым строк для каждой строки нашего датафрейма:
for (ColumnName, RowData) in df_final.head(1).iteritems():
    print('Имя столбца: \t\t', ColumnName)
    print('Содержимое строки \t\t:', RowData.values)

#### Сохраним результаты в CSV

In [ ]:
today

In [ ]:
file_name = '{}_HeadHunter_Data_Upload.csv'
file_name

In [ ]:
csv_file_name = file_name.format(today)
csv_file_name

In [ ]:
df_final.to_csv(csv_file_name, index=False)
print('OK! File {} is written.'.format(csv_file_name))

#### Проверим CSV


In [ ]:
file_path = './' + csv_file_name
file_path

In [ ]:
df_read = pd.read_csv(file_path)

In [ ]:
df_read

In [ ]:
df_read.info()

#### Начальный анализ наших данных


In [ ]:
df_read.describe()

In [ ]:
df_read.describe(include=[object]) 

In [ ]:
df_read.describe(exclude=[np.number])  

In [ ]:
# самые популярные наиманования вакансий
df_read\
    .groupby('name', as_index=False)\
    .agg({'id':'count'})\
    .sort_values('id', ascending = False)\
    .rename(columns={"name": "наименование вакансии", 
                     "id": "количество"})\
    .head(30)

In [ ]:
# Самые популярные по откликами вакансии
df_read.sort_values('responses_counters', ascending = False)\
    .head(10)

In [ ]:
# выберем тех работодателей кто при отклике ни вакансию просит нас пройти какой либо тест
df_read.query('has_test == True')

In [ ]:
# посмотрим ссылки на эти ванансии и просто для интереса глянем на тесты
# посмотрим что конкретно интресует компании ???
df_read.query('has_test == True').alternate_url
#df_read.query('has_test == True').url

## Получение подробных данных по каждой вакансии

1. **Просмотр вакансии**
	 - `GET /vacancies/{vacancy_id}` где `vacancy_id` – идентификатор вакансии. Вернёт подробную информацию по указанной вакансии.
	 - [Github](https://github.com/hhru/api/blob/master/docs/vacancies.md#item) документация
	 - [Ссылка на странный сайт](https://speca.io/speca/headhunter-api#methods_group)

In [ ]:
# вот тут у нас храняться инливилуальные ID Вакансии из предыдущего запроса поиска по парметрам
df_final.id.describe()

In [ ]:
id_list_vacancies = df_final.id.tolist()
id_list_vacancies

In [ ]:
# requests.get('https://api.hh.ru/vacancies/76940963?host=hh.ru').json()

In [ ]:
# for id in id_list_vacancies:
#     # формируем адрес запроса
#     url =f'https://api.hh.ru/vacancies/{id}?host=hh.ru' 
#     print(url)

In [ ]:
# # Получим кортеж с именем столбца и содержимым строк для каждой строки нашего датафрейма:
# for (ColumnName, RowData) in df_final.query('id == 76736984').iteritems():
#     print('Имя столбца: \t\t', ColumnName)
#     print('Содержимое строки \t\t:', RowData.values)

### сформируем наш запрос на уникальные вакансии

In [ ]:
url_HH_vacancies

In [ ]:
# Внимание - выполняется около 8 минут
unique_vacancy = pd.DataFrame()

for id_vacancy in tqdm(id_list_vacancies):
    # формируем адрес запроса
    url_HH_unique_vacancy_ID =f'https://api.hh.ru/vacancies/{id_vacancy}?host=hh.ru'
    
    # запрос по вакансии
    request_unique_vacancy_ID = requests.get(url=url_HH_unique_vacancy_ID)
    response_unique_vacancy_ID = request_unique_vacancy_ID.json()
    
    unique_vacancy = pd.concat([unique_vacancy, pd.DataFrame(pd.json_normalize(response_unique_vacancy_ID))])

In [ ]:
response_unique_vacancy_ID.keys()

In [ ]:
unique_vacancy

In [ ]:
list(unique_vacancy.columns)
# unique_vacancy.columns

In [ ]:
unique_vacancy.dtypes

In [ ]:
unique_vacancy.id.describe()

In [ ]:
unique_vacancy.describe()

In [ ]:
df_read.describe(include=[object]) 

In [ ]:
df_read.describe(exclude=[np.number])  

In [ ]:
# unique_vacancy[['key_skills']]

In [ ]:
# Получим кортеж с именем столбца и содержимым строк для каждой строки нашего датафрейма:
for (ColumnName, RowData) in unique_vacancy.iloc[[5]].iteritems():
    print('Имя столбца: \t\t', ColumnName)
    print('Содержимое строки \t\t:', RowData.values)

#### Распарсим наше описание

In [ ]:
list(unique_vacancy[['description']].iloc[5])

In [ ]:
unique_vacancy['description_new'] = unique_vacancy['description'].apply(lambda x: (re.sub(r'<.*?>', '', str(x))))

In [ ]:
list(unique_vacancy[['description_new']].iloc[5])

#### Посмотрим ключевые навыки

In [ ]:
list(unique_vacancy[['key_skills']].iloc[5])